# Webscraping the data

In [2]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt
import urllib3
import regex
import re
import os

#### HTML strings
I want the data and codebooks for full year consolidated data, population characteristics, employment variables, hospital inpatient stays, emergency room visits, & office-based medical provider visits. The pages look very similar, so once I have working code for the first set, I should be able to just edit it to get the rest. If I set it up as a function, I should be able to call the function on subsequent pages.

In [ ]:
#H233 = first dataset for full year consolidated data
# H233 data page: <a href="download_data_files_detail.jsp?cboPufNumber=HC-233">HC-233</a>
# H233 data .xlsx: <a href="../data_files/pufs/h233/h233xlsx.zip">ZIP</a>
# H233 codebook: <a href="download_data_files_codebook.jsp?PUFId=H233">HTML</a>
    # H233 codebook first variable(ACCELI42): <a href="download_data_files_codebook.jsp?PUFId=H233&amp;varName=ACCELI42">ACCELI42</a>
    # H233 codebook first description(ACCELI42):<font class="smallBlack" face="Arial">PERS ELIGIBLE FOR ACCESS SUPPLEMENT-R4/2</font>
        # value: <font class="smallBlack" face="Arial">-1 INAPPLICABLE</font>
        # unweighted(frequency): <font class="smallBlack" face="Arial">423</font>
        # weighted(frequency): <font class="smallBlack" face="Arial">3,683,511</font>


To get the data, I need to:
- Request the first page
- Get the html(BS)
- Pull out the links to the pages for 2011-2021
- drop them in a list
- pass that list to a function that requests each page
- Download the ZIP containing the xlsx

To get the codebooks I need to proceed from there and:
- request the codebook html page
- get the html
- pull out the name for each variable
- drop them in a list
- pull out the description for each variable
- drop them in a list
- merge those lists into a dataframe
From there, I will need to:
- get the links for each variable name
- request each page
- get the values from the value column
- drop them in a list
- get the values from the unweighted column
- drop them in a list
- get the values from the weighted value column
- drop them in a list
- merge those lists into a dataframe
Finally:
- join the variable dfs onto the var+description df in a way where the variable name and description are repeated for each value pertaining to them

Then I will be able to use the codebooks to substitute the codes in the data for meaningful variable names with descriptions. Hopefully. Lol

#### 5/31- I have discovered that python cannot open these xlsx files for some reason. 
#### Going to alter the code to retrieve only sas files.

### *Full Year Consolidated Data* (DUPLICATE OF POPULATION DATA; RETIRED)

In [ ]:
# request main page & check response
# URL = 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_results.jsp?cboDataYear=All&cboDataTypeY=1%2CHousehold+Full+Year+File&buttonYearandDataType=Search&cboPufNumber=All&SearchTitle=Consolidated+Data'

# response = requests.get(URL)

# response

In [ ]:
#convert to BeautifulSoup object & print readable HTML
# soup = BeautifulSoup(response.text)

# print(soup.prettify())

In [ ]:
#find the table row and print the link for each PUF

# # link to data page for first year, H233: <a href="download_data_files_detail.jsp?cboPufNumber=HC-233">HC-233</a>
# # HTML path for the link: 
# # <table role= "presentation" | <tbody> | <tr> | <tdwidth = "584" | tablesummary = "PUF Search Results" | <tbody> |
# # <tr id="Row1_neoTD3" | <th scope = "row" | <div align="left"> |
# # <a href="download_data_files_detail.jsp?cboPufNumber=HC-233">HC-233</a></div>

# htmllinks = soup.findAll('tr', attrs = {'id' : "Row1_neoTD3"})
# # print(htmllinks)

# for href in htmllinks:
#     link = href.find('a').get('href')
#     print(link)

# # for href in htmllinks:
# #     re.search

# # links = []

# # for link in htmllinks:
# #     links.append(link.text)

# # print(links)
# # type(links)

In [6]:
# define function to retrieve the links from the page(collapse above)

def getPUFLinks(URL):
    response = requests.get(URL)
    soup = BeautifulSoup(response.text)
    PUFlinks = []

    for link in soup.findAll('tr', attrs = {'id' : 'Row1_neoTD3'}):
        href = link.find('a').get('href')
        PUF = href.split('=')[1]
        if PUF in PUFids:
            PUFlinks.append(link.find('a').get('href'))

    return PUFlinks

In [ ]:
#get the links for full consolidated data pages
# PUFids = ['HC-233', 'HC-224', 'HC-216', 'HC-209', 'HC-201', 'HC-192', 'HC-181', 'HC-171', 'HC-163', 'HC-155']
# URL = 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_results.jsp?cboDataYear=All&cboDataTypeY=1%2CHousehold+Full+Year+File&buttonYearandDataType=Search&cboPufNumber=All&SearchTitle=Consolidated+Data'

# fcd = getPUFLinks(URL)
# print(fcd)

In [ ]:
#create list of links to get pages with .zips

# PUFlinks = (['https://meps.ahrq.gov/mepsweb/data_stats/' + word for word in fcd])
# print(PUFlinks)


In [9]:
#define function to get soup of pages with .zips and get the .zip links

# def getZIPLinks(URL):
#     response = requests.get(URL)
#     soup = BeautifulSoup(response.text)
#     ZIPlinks = []
    
#     for link in soup.findAll('tr', attrs = {'id' : 'faqRoll_neoTD3'}):
#         href = link.find('a').get('href')
#         suffix = 'xlsx.zip'
#         if href.endswith(suffix) == True:
#             ZIPlinks.append(link.find('a').get('href'))

#     return ZIPlinks

In [ ]:
#get the links to the .zips

# ZIPids= []

# for link in PUFlinks:
#     ZIPids.extend(getZIPLinks(link))
# ZIPids

Oh no, there are only .xlsx for five of my ten chosen years. only ASCII and SAS after that. conundrum (T_T)

##### Update: xlsx nonviable anyway. proceed with SAS!

In [13]:
#alter function to retrieve SAS links
def getSASLinks(URL):
    response = requests.get(URL)
    soup = BeautifulSoup(response.text)
    SASlinks = []
    
    for link in soup.findAll('tr', attrs = {'id' : 'faqRoll_neoTD3'}):
        href = link.find('a').get('href')
        suffix = 'ssp.zip'
        if href.endswith(suffix) == True:
            SASlinks.append(link.find('a').get('href'))

    return SASlinks

##### Update: some SAS files also nonviable. proceed with SASv9!

In [39]:
#alter function to retrieve SASv9 links
def getV9Links(URL):
    response = requests.get(URL)
    soup = BeautifulSoup(response.text)
    SASlinks = []
    
    for link in soup.findAll('tr', attrs = {'id' : 'faqRoll_neoTD3'}):
        href = link.find('a').get('href')
        suffix = 'v9.zip'
        if href.endswith(suffix) == True:
            SASlinks.append(link.find('a').get('href'))

    return SASlinks

In [ ]:
#get the links to the SAS files

# SASids = []

# for link in PUFlinks:
#     SASids.extend(getV9Links(link))
# SASids

In [ ]:
#remove the ones i have xlsx for already

# SASids = SASids[5:]
# SASids

In [ ]:
#put all ids in one list to run function
# allids = ZIPids + SASids
# allids

In [15]:
#define function to download the linked data files
def dlFiles(list, dir):
    try:
        if not os.path.exists(dir):                                        #check if directory exists, if not, create it
            os.makedirs(dir)
        for url in list:
            l= []                                                         #create empty list and use to append the url prefix to the file url
            prefix = 'https://meps.ahrq.gov/mepsweb/data_stats/'
            l.append(prefix)
            l.append(url)
            url = ''.join(l)
            response = requests.get(url)                                  #get url response and check if successful
            response.raise_for_status()
            filename = os.path.join(dir, os.path.basename(url))           #create filename using string from url
            with open(filename, 'wb') as file:                            #open file in binary mode to write to it
                file.write(response.content)
    except requests.exceptions.HTTPError as e:                            #errors to return if access fails
        print(f'HTTP error occurred for {url}: {e}')
    except requests.exceptions.RequestException as e:
        print(f'Error occurred for {url}: {e}')
         

In [44]:
#call function to download files
# list = SASids
# dir = '../data/fcd'
# dlFiles(list, dir)

### *Full Year Population Characteristics*

In [45]:
URL = 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_results.jsp?cboDataYear=All&cboDataTypeY=1%2CHousehold+Full+Year+File&buttonYearandDataType=Search&cboPufNumber=All&SearchTitle=Population+Characteristics'
PUFids = ['HC-233', 'HC-224', 'HC-216', 'HC-209', 'HC-201', 'HC-192', 'HC-181', 'HC-171', 'HC-163', 'HC-155']

In [46]:
# get page link text
fpc = getPUFLinks(URL)

In [47]:
#turn link text into full links to pages

PUFlinks = (['https://meps.ahrq.gov/mepsweb/data_stats/' + word for word in fpc])
print(PUFlinks)

['https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-233', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-224', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-216', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-209', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-201', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-192', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-181', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-171', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-163', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-155']


In [ ]:
#get the links to the xlsx.zips

# ZIPids= []

# for link in PUFlinks:
#     ZIPids.extend(getZIPLinks(link))
# ZIPids

In [20]:
#get the links to the SAS files

SASids = []

for link in PUFlinks:
    SASids.extend(getSASLinks(link))
SASids

['../data_files/pufs/h233/h233ssp.zip',
 '../data_files/pufs/h224/h224ssp.zip',
 '../data_files/pufs/h216/h216ssp.zip',
 '../data_files/pufs/h209/h209ssp.zip',
 '../data_files/pufs/h201/h201ssp.zip',
 '../data_files/pufs/h192ssp.zip',
 '../data_files/pufs/h181ssp.zip',
 '../data_files/pufs/h171ssp.zip',
 '../data_files/pufs/h163ssp.zip',
 '../data_files/pufs/h155ssp.zip']

In [48]:
#get the links to the V9 files

SASids = []

for link in PUFlinks:
    SASids.extend(getV9Links(link))
SASids

['../data_files/pufs/h233/h233v9.zip',
 '../data_files/pufs/h224/h224v9.zip',
 '../data_files/pufs/h216/h216v9.zip',
 '../data_files/pufs/h209/h209v9.zip',
 '../data_files/pufs/h201/h201v9.zip']

In [ ]:
#remove the ones i have xlsx for already

# SASids = SASids[5:]
# SASids

In [ ]:
#put all ids in one list
# allids = ZIPids + SASids
# allids

In [49]:
#call function to download files
list = SASids
dir = '../data/fpc'
dlFiles(list, dir)

### *Hospital Inpatient Stays*

In [65]:
URL = 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_results.jsp?cboDataYear=All&cboDataTypeY=2%2CHousehold+Event+File&buttonYearandDataType=Search&cboPufNumber=All&SearchTitle=Hospital+Inpatient+Stays'
PUFids = ['HC-229D', 'HC-220D', 'HC-213D', 'HC-206D', 'HC-197D', 'HC-188D', 'HC-178D', 'HC-168D', 'HC-160D', 'HC-152D']

In [66]:
# get page link text
his = getPUFLinks(URL)

In [67]:
#turn link text into full links to pages

PUFlinks = (['https://meps.ahrq.gov/mepsweb/data_stats/' + word for word in his])
print(PUFlinks)

['https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-229D', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-220D', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-213D', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-206D', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-197D', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-188D', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-178D', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-168D', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-160D', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-152D']


In [53]:
#get the links to the xlsx.zips

# ZIPids= []

# for link in PUFlinks:
#     ZIPids.extend(getZIPLinks(link))
# ZIPids

In [68]:
#get the links to the SAS files

SASids = []

for link in PUFlinks:
    SASids.extend(getSASLinks(link))
SASids

['../data_files/pufs/h229d/h229dssp.zip',
 '../data_files/pufs/h220d/h220dssp.zip',
 '../data_files/pufs/h213d/h213dssp.zip',
 '../data_files/pufs/h206d/h206dssp.zip',
 '../data_files/pufs/h197d/h197dssp.zip',
 '../data_files/pufs/h188dssp.zip',
 '../data_files/pufs/h178dssp.zip',
 '../data_files/pufs/h168dssp.zip',
 '../data_files/pufs/h160dssp.zip',
 '../data_files/pufs/h152dssp.zip']

In [54]:
#get the links to the V9 files

SASids = []

for link in PUFlinks:
    SASids.extend(getV9Links(link))
SASids

['../data_files/pufs/h229d/h229dv9.zip',
 '../data_files/pufs/h220d/h220dv9.zip',
 '../data_files/pufs/h213d/h213dv9.zip',
 '../data_files/pufs/h206d/h206dv9.zip',
 '../data_files/pufs/h197d/h197dv9.zip']

In [ ]:
#remove the ones i have xlsx for already

# SASids = SASids[5:]
# SASids

In [ ]:
#put all ids in one list
# allids = ZIPids + SASids
# allids

In [55]:
#call function to download files
list = SASids
dir = '../data/his'
dlFiles(list, dir)

### *Emergency Room Visits*

In [56]:
URL = 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_results.jsp?cboDataYear=All&cboDataTypeY=2%2CHousehold+Event+File&buttonYearandDataType=Search&cboPufNumber=All&SearchTitle=Emergency+Room+Visits'
PUFids = ['HC-229E', 'HC-220E', 'HC-213E', 'HC-206E', 'HC-197E', 'HC-188E', 'HC-178E', 'HC-168E', 'HC-160E', 'HC-152E']

In [57]:
# get page link text
erv = getPUFLinks(URL)

In [58]:
#turn link text into full links to pages

PUFlinks = (['https://meps.ahrq.gov/mepsweb/data_stats/' + word for word in erv])
print(PUFlinks)

['https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-229E', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-220E', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-213E', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-206E', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-197E', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-188E', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-178E', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-168E', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-160E', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-152E']


In [ ]:
#get the links to the xlsx.zips

# ZIPids= []

# for link in PUFlinks:
#     ZIPids.extend(getZIPLinks(link))
# ZIPids

In [ ]:
#get the links to the SAS files

SASids = []

for link in PUFlinks:
    SASids.extend(getSASLinks(link))
SASids

In [59]:
#get the links to the V9 files

SASids = []

for link in PUFlinks:
    SASids.extend(getV9Links(link))
SASids

['../data_files/pufs/h229e/h229ev9.zip',
 '../data_files/pufs/h220e/h220ev9.zip',
 '../data_files/pufs/h213e/h213ev9.zip',
 '../data_files/pufs/h206e/h206ev9.zip',
 '../data_files/pufs/h197e/h197ev9.zip']

In [ ]:
#remove the ones i have xlsx for already

# SASids = SASids[5:]
# SASids

In [ ]:
# #put all ids in one list
# allids = ZIPids + SASids
# allids

In [60]:
#call function to download files
list = SASids
dir = '../data/erv'
dlFiles(list, dir)

### *Office-Based Medical Provider Visits*

In [33]:
URL = 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_results.jsp?cboDataYear=All&cboDataTypeY=2%2CHousehold+Event+File&buttonYearandDataType=Search&cboPufNumber=All&SearchTitle=Office-Based+Medical+Provider+Visits'
PUFids = ['HC-229G', 'HC-220G', 'HC-213G', 'HC-206G', 'HC-197G', 'HC-188G', 'HC-178G', 'HC-168G', 'HC-160G', 'HC-152G']

In [34]:
# get page link text
obv = getPUFLinks(URL)

In [35]:
#turn link text into full links to pages

PUFlinks = (['https://meps.ahrq.gov/mepsweb/data_stats/' + word for word in obv])
print(PUFlinks)

['https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-229G', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-220G', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-213G', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-206G', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-197G', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-188G', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-178G', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-168G', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-160G', 'https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-152G']


In [ ]:
#get the links to the xlsx.zips

# ZIPids= []

# for link in PUFlinks:
#     ZIPids.extend(getZIPLinks(link))
# ZIPids

In [ ]:
#get the links to the SAS files

SASids = []

for link in PUFlinks:
    SASids.extend(getSASLinks(link))
SASids

In [61]:
#get the links to the V9 files

SASids = []

for link in PUFlinks:
    SASids.extend(getV9Links(link))
SASids

['../data_files/pufs/h229e/h229ev9.zip',
 '../data_files/pufs/h220e/h220ev9.zip',
 '../data_files/pufs/h213e/h213ev9.zip',
 '../data_files/pufs/h206e/h206ev9.zip',
 '../data_files/pufs/h197e/h197ev9.zip']

In [ ]:
#remove the ones i have xlsx for already

# SASids = SASids[5:]
# SASids

In [ ]:
#put all ids in one list
# allids = ZIPids + SASids
# allids

In [62]:
#call function to download files
list = SASids
dir = '../data/obv'
dlFiles(list, dir)